In [1]:
import re
import numpy as np
import os
os.sys.path.append('../1/')
from z2 import loader
from math import log
import sys
import heapq
import collections
import operator


vowels = list('aeioóuyąę') + list('aeioóuyąę'.upper())
compacted_vovels = ['i' + x for x in vowels if x != 'i']
word2tag = dict()
tag2word = dict()


def stringNorm(sent, num=False):
    regex = re.compile(f'[,\.!?:;\'{"0-9" if not num else ""}\*\-“…\(\)„”—»«–––=\[\]’]')
    return regex.sub('',sent.lower())

def bigrams2unigrams(bigrams):
    return {w1: sum([float(bigrams[w1][w2]) for w2 in bigrams[w1]])/2 for w1 in bigrams}

def count_syllable(phrase, verose=False):
    res = 0
    for i, letter in enumerate(phrase):
        if letter in vowels:
            res += 1
            if verose:
                print(letter)
        if phrase[i:i+2] in compacted_vovels:
            res -= 1
            if verose:
                print(phrase[i:i+2])
    return res


with open("data/supertags.txt") as tags:
    for line in tags:
        word, tag = stringNorm(line, num=True).split()
        word2tag[word] = tag
        if tag in tag2word:
            tag2word[tag].append(word)
        else:
            tag2word[tag] = [word]
            

bigrams = loader('../1/poleval_2grams.txt', cut = 10)
rev_bigram = {w2:{w1:bigrams[w1][w2] } for w1 in bigrams for w2 in bigrams[w1] }
unigrams = bigrams2unigrams(bigrams)
safeGrams = {x for x in rev_bigram}.intersection({y for y in bigrams})
uniSum = sum([float(unigrams[x]) for x in unigrams])

with open('data/pan-tadeusz.txt') as f:
    pt = [stringNorm(x).split() for x in f if  count_syllable(stringNorm(x)) == 13]
    
acents = collections.Counter([
    tuple([count_syllable(y) for y in x])
    for x in pt
])

PMI = lambda w1, w2: log(
    float(bigrams[w1][w2] if w1 in bigrams and w2 in bigrams[w1] else 1) * uniSum / (unigrams[w1] * unigrams[w2]) 
    + sys.float_info.min)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000


In [2]:

def get_rym(w):
    best = None
    for i in range(len(w)):
        if count_syllable(w[i:]) == 2:
            best = w[i:]
    return best

def sample_verset():
    try:
        index = np.random.choice(np.arange(len(pt)))
        rym = get_rym(pt[index][-1])
        if get_rym(pt[index + 1][-1]) == rym:
            return pt[index], pt[index + 1]
        elif get_rym(pt[index - 1][-1]) == rym:
            return pt[index], pt[index - 1]
        return sample_verset()
    except:
        return sample_verset()
    

def get_accents(phrase):
    return [count_syllable(x) for x in phrase]


In [3]:
def sameTags(w):
    if w in word2tag:
        return tag2word[word2tag[w]]
    elif ('^' + w)[-3:] in word2tag:
        return tag2word[word2tag[('^' + w)[-3:]]]
    else:
        raise Exception(f"***niestety nie ma {w} i {('^' + w)[-3:]}***\n")
        x = np.random.choice(list(tag2word),1)
        return tag2word[x[0]]
    
def createAltWords(accent, verse, rime=None):
    return [list(
        set(
            filter(
                lambda x: count_syllable(x) == accent[i],
                sameTags(w)
            )
        ).intersection(
            {y for y in safeGrams}
        ))
        for i, w in enumerate(verse)
    ]

def getProb2(word, prev):
    if word in bigrams[prev]:
#         print('aa: ', bigrams[prev][word])
        return bigrams[prev][word]
    return 0.0001 * unigrams[word]

def getRandWord(w):
    p = np.array([unigrams[x] for x in w])
    p = p/np.sum(p)
    
    if len(w) > 0:
        choosen = np.random.choice(w, 1, p=p)
        return choosen[0]
    else:
        raise Exception("Pusto")

def getRandWord2Rev(words, _next):
    probs = np.array([getProb2(_next, x) for x in words]).astype(float)
    probs = probs / np.sum(probs)
    return str(np.random.choice(words, 1, p=probs)[0])

def generateAccent(k=1):
    p = np.array([acents[x] for x in acents])
    p = p/np.sum(p)
    accs = np.random.choice(len(list(acents)),k, p=p)
    return (list(acents)[x] for x in accs)

In [4]:
licznik = 0
while True:
    try:
        v1, v2 = sample_verset()
        a1, a2 = get_accents(v1), get_accents(v2)
#         a1, a2 = generateAccent(2)
#         print(a1, a2)
        
        alt1 = createAltWords(a1,v1)
        sent1 = []
        old = getRandWord(alt1[-1])
        sent1.append(old)

        for w in reversed(alt1[:-1]):
            choosen = getRandWord2Rev(w, old)
            sent1.append(choosen)
            old = choosen

        sent1.reverse()

        alt2 = createAltWords(a2,v2)
        alt2[-1] = list(filter(lambda x: get_rym(x) == get_rym(sent1[-1]), alt2[-1]))
        sent2 = []
        
        if len(alt2[-1]) < 2 and alt2[-1][0] == sent1[-1]:
            raise Exception("Ten sam wyraz")
        else:
            old = getRandWord(alt2[-1])
            while old == sent1[-1]:
                old = getRandWord(alt2[-1])
                
        sent2.append(old)

        for w in reversed(alt2[:-1]):
            choosen = getRandWord2Rev(w, old)
            sent2.append(choosen)
            old = choosen
        sent2.reverse()   

        print(' '.join(v1))
        print(' '.join(v2))
        print()
        print(' '.join(sent1))
        print(' '.join(sent2))
        break
    except:
        licznik += 1
        print(f"Próba nr: {licznik}")

Próba nr: 1
Próba nr: 2
Próba nr: 3
Próba nr: 4
Próba nr: 5
Próba nr: 6
Próba nr: 7
Próba nr: 8
Próba nr: 9
Próba nr: 10
Próba nr: 11
Próba nr: 12
Próba nr: 13
Próba nr: 14
Próba nr: 15
już zraniono i poległ jeden z bartłomiejów
ledwie który zabije przecież dwóch maciejów

i wdrożono i opadł jeden z zawodników
jako jeden wytworzy sobie dwóch rolników


In [5]:
"""
że dotychczas podjęcia badań jakościowych
niemal pionowe wsparcia przedsiębiorstw państwowych
"""

'\nże dotychczas podjęcia badań jakościowych\nniemal pionowe wsparcia przedsiębiorstw państwowych\n'

In [40]:
licznik = 0
tries = 25


while True:
    try:
        wrongs = 0
        it = 0
        sentTab = []
        v1, v2 = sample_verset()
        a1, a2 = get_accents(v1), get_accents(v2)
#         a1, a2 = generateAccent(2)
#         print(a1, a2)
        while it < tries:
            if wrongs > 3 + 2 * it:
                raise Exception('Za dużo złych')
            try:
                alt1 = createAltWords(a1,v1)
                sent1 = []
                old = getRandWord(alt1[-1])
                sent1.append(old)

                for w in reversed(alt1[:-1]):
                    choosen = getRandWord2Rev(w, old)
                    sent1.append(choosen)
                    old = choosen

                sent1.reverse()

                alt2 = createAltWords(a2,v2)
                alt2[-1] = list(filter(lambda x: get_rym(x) == get_rym(sent1[-1]), alt2[-1]))
                sent2 = []

                if len(alt2[-1]) < 2 and alt2[-1][0] == sent1[-1]:
                    raise Exception("Ten sam wyraz")
                else:
                    old = getRandWord(alt2[-1])
                    while old == sent1[-1]:
                        old = getRandWord(alt2[-1])

                sent2.append(old)

                for w in reversed(alt2[:-1]):
                    choosen = getRandWord2Rev(w, old)
                    sent2.append(choosen)
                    old = choosen
                sent2.reverse()
                sentTab.append((sent1, sent2),)
                it+=1
                print('.', end='', flush=True)
            except:
                print('x', end='', flush=True)
                wrongs+=1

#         print(' '.join(v1))
#         print(' '.join(v2))
#         print()
#         print(' '.join(sent1))
#         print(' '.join(sent2))
        break
    except:
        licznik += 1
        print(f"\nPróba nr: {licznik}")
        
values = {}
for sent1, sent2 in sentTab:
#     sent1.extend(sent2)
    sent = sent1 + sent2
    val = 0
    for w1, w2 in zip(sent, sent[1:]):
        val+= PMI(w1, w2)
    s = ' '.join(sent1)+'\n' + ' '.join(sent2)
    values[s] = val
    
sorted_values = sorted(values.items(), key=operator.itemgetter(1))
for sent, value in sorted_values:
    print('\n', sent, '\n', value)

xxxx
Próba nr: 1
xxxx
Próba nr: 2
xxxx
Próba nr: 3
xxxx
Próba nr: 4
x...x.x.x.......x...x..........
 jak zawodnicy wzięli agencję żydowską
na prezenty mimo że już saługę lwowską 
 43.48862853288023

 kret ochroniarze wznieśli ideę sowiecką
na przekazy wyżej za i alicję grecką 
 48.307500956153156

 jak zawodnicy wzięli opinię publiczną
i klasztory frotte no i głupotę liczną 
 51.34394369125595

 kszyk czarodzieje zdjęli parafię poważną
i etapy ale i o obrazę ważną 
 53.49375052059631

 smok misjonarze wzięli przysięgę wojskową
i sufity społem się na demencję nową 
 53.840706098596165

 ssak lokatorzy wznieśli współpracę wojskową
ho serniki bardzo się i blokadę nową 
 55.11293424338906

 krab amatorzy wsparli wykładnię kamienną
o powody ale się za niceę zmienną 
 55.71771531256858

 wąż podróżnicy wzięli krytykę społeczną
i tunele cary się i dolinę rzeczną 
 58.81454405604046

 pstrąg delegaci wznieśli atencję śluzową
i miotacze bardziej że przez milicję nową 
 59.80349419405519

 żółw 